# PRÁCTICA 2: Clasificador de noticias

### Nombres:
Introduce en esta celda los nombres de los dos integrantes del grupo:
- *Alumno 1:* DANIEL CARMONA PEDRAJAS
- *Alumno 2:* JOEL PARDO FERRERA

Objetivo: Implementar un clasificador usando el conjunto de datos recopilado de varias fuentes de internet como:

- Google News que toma noticias de varios repositorios dedicados a la información
- Periódicos:
    - El País
    - ABC
    - El Confidencial
    - 20minutos
    - El Diario

Este repositorio incluye tanto las noticias en formato '.txt' donde se almacenan los cuerpos de noticia y sus correspondientes títulos, como un '.csv' donde se contiene un registro de todas las noticias donde se refleja el número de noticias, la clase a la que pertenece (deportes, salud, ciencia y politica), el número de noticia dentro de la clase correspondiente, el título de noticia, la ruta donde está almacenada esa noticia, y por último la URL de donde se ha sacado la noticia. 

La fechas tanto de publicación como de obtención e datos se ubican en Noviembre de 2022. 

La clase a predecir es el tipo de noticia (columna 'category' de la base de datos), a partir de los archivos '.txt'. 

## 1. IMPORTACIÓN DE LIBRERÍAS

In [1]:
# git config --global user.email "jpardo0824@gmail.com"
# git config --global user.name "JPardo08"

In [2]:
# !pip3 install pandas
# !pip3 install spacy
# !pip3 install scikit-learn

In [3]:
import pandas as pd
import numpy as np
import spacy
import unicodedata
import os
#from spellchecker import SpellChecker 
#from textblob import TextBlob 
#import contractions
import re
import random
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC

## 2. CARGA DE DATOS
Cargamos los datos en dos formatos:
* DataFrame de pandas
* Generador

Para cargar los datos utilizamos la librería pandas. 
La funcion implementada recibe la ruta del archivo .csv que queramos cargar y devuelve los textos en una variable generador.

In [4]:
ruta = ".." ## CAMBIAR

In [5]:
def cargar_txts(dataframe):
    """ 
    Función para coger los documentos '.txt' de las noticias
    """
    paths = dataframe["path"].tolist()
    clases = dataframe["category"].unique().tolist()

    documentos = []

    for p in paths:
        t = p.replace(".", ruta, 1) 
        s = t.replace("/", "//")
        print(s)

        with open(s, "r", encoding="latin-1", errors='ignore') as f:
            lineas = f.readlines()
            txt1 = ''.join(lineas)
            documentos.append(txt1)
            
    return clases, documentos

In [7]:
urls_dataframe = ruta + "/Datos/urls1.csv"
path_urls_dataframe = urls_dataframe.replace("/", "//")
print(path_urls_dataframe)

# df_gen = parse(r'/Users/joelpardo/Desktop/TextClassification/Datos/urls1.csv')



noticias_dataframe = pd.read_csv(path_urls_dataframe,',')

#Visualizamos los datos
noticias_dataframe.head()

..//Datos//urls1.csv


C:\Users\Daniel\AppData\Local\Temp\ipykernel_18128\2303741000.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  noticias_dataframe = pd.read_csv(path_urls_dataframe,',')


,index,category,n_doc,title,path,link
0,0,sports,1,Memphis y Dumfries certifican que el 'soccer' ...,./Datos/Raw_data/sports/1.txt,https://news.google.com/__i/rss/rd/articles/CB...
1,1,sports,2,Con el dinero no basta en el fútbol - AS,./Datos/Raw_data/sports/2.txt,https://news.google.com/__i/rss/rd/articles/CB...
2,2,sports,3,El fútbol es un cuento - La Voz de Galicia,./Datos/Raw_data/sports/3.txt,https://news.google.com/__i/rss/rd/articles/CB...
3,3,sports,4,Cavani y Giménez serán sancionados pero no con...,./Datos/Raw_data/sports/4.txt,https://news.google.com/__i/rss/rd/articles/CB...
4,4,sports,5,Suspendido por una tangana a puñetazos el Alme...,./Datos/Raw_data/sports/5.txt,https://news.google.com/__i/rss/rd/articles/CB...


In [9]:
clases, documentos = cargar_txts(noticias_dataframe)
print(len(clases))
print(len(documentos))

..//Datos//Raw_data//sports//1.txt
..//Datos//Raw_data//sports//2.txt
..//Datos//Raw_data//sports//3.txt
..//Datos//Raw_data//sports//4.txt
..//Datos//Raw_data//sports//5.txt
..//Datos//Raw_data//sports//6.txt
..//Datos//Raw_data//sports//7.txt
..//Datos//Raw_data//sports//8.txt
..//Datos//Raw_data//sports//9.txt
..//Datos//Raw_data//sports//10.txt
..//Datos//Raw_data//sports//11.txt
..//Datos//Raw_data//sports//12.txt
..//Datos//Raw_data//sports//13.txt
..//Datos//Raw_data//sports//14.txt
..//Datos//Raw_data//sports//15.txt
..//Datos//Raw_data//sports//16.txt
..//Datos//Raw_data//sports//17.txt
..//Datos//Raw_data//sports//18.txt
..//Datos//Raw_data//sports//19.txt
..//Datos//Raw_data//sports//20.txt
..//Datos//Raw_data//sports//21.txt
..//Datos//Raw_data//sports//22.txt
..//Datos//Raw_data//sports//23.txt
..//Datos//Raw_data//sports//24.txt
..//Datos//Raw_data//sports//25.txt
..//Datos//Raw_data//sports//26.txt
..//Datos//Raw_data//sports//27.txt
..//Datos//Raw_data//sports//28.txt
.

Comprobamos los tipos de datos de las columnas de nuestro dataframe

In [10]:
print(type(noticias_dataframe))
# print(type(df_gen))

<class 'pandas.core.frame.DataFrame'>


In [11]:
print(documentos[0])

El lateral neerlandés regaló dos asistencias, marcó el 1-3... y el delantero azulgrana bailó con la defensa estadounidense para clasificar a la Oranje a los cuartos de final del Mundial
"En Países Bajos aprendí mucho, tomé muchos conceptos de fútbol". Gregg Berhalter, seleccionador de Estados Unidos, aseguró en la previa del partido que se inspiró en el fútbol neerlandés para fijar las bases de su ideario como entrenador. Unos conceptos futbolísticos que se dejaron ver en los primeros compases del partido, hasta que Memphis Depay decidió romper el espejismo impuesto por la caballería yankee.
Averiguaron la forma en la que tenían que atacar a Países Bajos para desarmar su entramado defensivo pero Pulisic, que no tuvo su día, no consiguió mandar al fondo de la red ni una de las dos ocasiones que tuvo en la primera parte. Una de ellas tardará mucho tiempo en olvidarla. 
Instaurados en el bloque bajo, los hombres de Louis van Gaal dieron una auténtica lección de cómo un equipo tiene que at

In [12]:
noticias_dataframe["docs"] = documentos

In [13]:
#Miramos los nombres
noticias_dataframe.columns

Index(['index', 'category', 'n_doc', 'title', 'path', 'link', 'docs'], dtype='object')

Generamos un duplicado de nuestro dataframe para reducir el tamaño y trabajar de forma más cómoda

In [15]:
noticias_dataframe2 = noticias_dataframe.loc[:,['title', 'category','docs']]
noticias_dataframe2=noticias_dataframe2.dropna()

Comprobamos que el número de instancias sigue siendo el mismo

In [17]:
print(noticias_dataframe.shape)
print(noticias_dataframe2.shape)

(240, 7)
(240, 3)


Mostramos las posibles categorías de nuestros textos

In [20]:
clases = pd.unique(noticias_dataframe2['category']).tolist()
print(clases)


['sports', 'health', 'science', 'politics']


Y comprobamos si las categorías están balanceadas en cuanto a peso.

In [23]:
frecuencias = noticias_dataframe2.groupby('category').count()
frecuencias = frecuencias.sort_values('title',ascending=False)

frecuencias.head()

,title,docs
category,,
health,60,60
politics,60,60
science,60,60
sports,60,60


Como podemos observar, todas las clases tienen el mismo peso, es decir, tienen el mismo numero de noticas. Tenemos un conjunto de datos homogeneo para el cual hacer la prediccion.

## 3. PRE-PROCESADO
Realizaremos el siguiente procesado:
- Separar el texto en *tokens*
- Eliminar los *tokens* de tipo *stop-word*, signos de puntuación, signos especiales o espacios
- Lematizar el texto
- Introducimos un espacio después de determinados signos de puntuación (".", "?", "%") para que el tokenizado sea correcto
- Filtramos los *tokens* con una longitud de 1



In [ ]:
!python -m spacy download es_core_news_lg

In [ ]:
# https://spacy.io/models/es
nlp = spacy.load("es_core_news_lg") #Mejor modelo optimizado para la CPU

#definimos función de normalizado
def normaliza(texto):
    texto = re.sub(r"(\.)|(\?)|(\%)", r"\1 |\2 |\3 ", texto) #añadimos un espacio después de "." y "?"
    doc = nlp(texto)
    tokens = [t for t in doc if
                        len(t.text)>1 and
                       not t.is_stop and
                       not t.is_space and
                       not t.is_punct]#filtramos los tokens que nos interesan
    palabras = []
    for t in tokens:
        palabras.append(t.lemma_) #añadimos lema
    salida = ' '.join(palabras)#junta todos los tokens en un string
    
    return salida

#funcion para quitar acentos y caracteres no ASCII:

def remove_accents(input_str):     
    nfkd_form = unicodedata.normalize('NFKD', input_str)     
    only_ascii = nfkd_form.encode('ASCII', 'ignore')     
    return only_ascii.decode("utf-8", 'ignore')


In [ ]:
#Comprobamos que funciona con el elemento 0 de noticia ("noticia")
print(df2['docs'][0])
print("-"*140)
normaliza(df2['docs'][0])

In [ ]:
#Comprobamos que funciona con el elemento 2 de noticia ("docs")
print(df2['docs'][2])
print("-"*140)
normaliza(df2['docs'][2])

In [ ]:
#Creacion de corpus y labels 
df2["corpus"] = df2['docs'] + df2['title']
corpus = df2["corpus"].tolist()

labels_posibles = pd.unique(df2['category']).tolist()
labels = df2['category'].tolist()

In [ ]:
prediccion_r=ruta+"/Prediccion"

os.mkdir(prediccion_r)

df2.to_csv(prediccion_r+'/training.csv', header=True, index=False)

In [ ]:
#Vemos cuantas observaciones en total tenemos.
print(f'\nTamaño  TOTAL: {len(corpus)}')
print(f'Tamaño de clase TOTAL: {len(labels)}')


In [ ]:
#Seleccionamos 5000 muestras del cojunto de datos total
# corpus_sm, labels_sm = resample(corpus, labels, n_samples=5000, replace=False) 

In [ ]:
#Separacion del conjunto de datos P5
train_corpus, test_corpus, train_labels, test_labels = train_test_split(corpus, labels, test_size = 0.30, shuffle = False)

In [ ]:
# COMPLETAR
print(f'\nTamaño de TRAIN: {len(train_corpus)}')
print(f'Tamaño de clase TRAIN: {len(train_labels)}')

print(f'\nTamaño de TEST: {len(test_corpus)}')
print(f'Tamaño de clase TEST: {len(test_labels)}')

In [ ]:
#Normalizamos los docss y los guardamos como una lista en lugar de generator porque 
#lo tenemos que múltiples veces y no queremos tener que normalizar todo el corpus cada vez.
norm_train_corpus = list(map(normaliza, train_corpus))

In [ ]:
norm_test_corpus = list(map(normaliza, test_corpus))

## 4. EXTRACCION DE CARACTERISTICAS
Instanciamos los vectorizadores para obtener las características BoW y TF-IDF.  
Usamos el parámetro max_df=0.9 para eliminar los stop-words como las palabras que aparecen al menos en el 90% de los documentos y el parámetro min_df=0.01 para eliminar las palabras que no aparecen al menos en un 1% de los documentos.\

Usamos el modelo `TfidfTransformer` para calcular la matriz TF-IDF a partir del BoW y no tener que repetir todo el entrenamiento.

Para calcular los modelos basados en WV usamos el modelo gloVe pre-entrenado en `spaCy`. Calculamos dos modelos basados en word-vectors:  
* El vector promedio de los WV de todos los tokens con el mismo peso para todas las palabras.  
* Ponderando el WV de cada palabra por el término de frecuencia inversa de documento (IDF).  

Definimos las funciones para calcular estas dos matrices de características:

In [ ]:
#BoW
bow_vectorizer = CountVectorizer(min_df=0.01, max_df=0.9)

#Tf-idf
tfidf_vectorizer = TfidfTransformer()

#Funciones de WV.
def averaged_word_vectorizer(corpus):
    '''Aplica la función de cálculo del WE promedio a todos los
    documentos del corpus (cada doc es una lista de tokens)'''
    features = [nlp(doc).vector
                    for doc in corpus]
    return np.array(features)

def tfidf_wtd_avg_word_vectors(doc, word_tfidf_map):
    '''Aplica la función de cálculo del WE ponderado por TF-IDF
    a un documento (como lista de tokens)'''
    tokens = doc.split()

    feature_vector = np.zeros((nlp.vocab.vectors_length,),dtype="float64")
    wts = 0.      
    for word in tokens:
        if nlp.vocab[word].has_vector and word_tfidf_map.get(word, 0): #sólo considera palabras conocidas
            weighted_word_vector = word_tfidf_map[word] * nlp.vocab[word].vector
            wts = wts + 1
            feature_vector = np.add(feature_vector, weighted_word_vector)
    if wts:
        feature_vector = np.divide(feature_vector, wts)
        
    return feature_vector
    
def tfidf_weighted_averaged_word_vectorizer(corpus, word_tfidf_map):
    '''Aplica la función de cálculo del WE ponderado por TF-IDF a todos los
    documentos del corpus (cada doc es una lista de tokens)'''                                       
    features = [tfidf_wtd_avg_word_vectors(doc, word_tfidf_map)
                    for doc in corpus]
    return np.array(features)

In [ ]:
# características bag of words
bow_train_features = bow_vectorizer.fit_transform(norm_train_corpus)  
bow_test_features = bow_vectorizer.transform(norm_test_corpus) 

# características tfidf (a partir del BoW)
tfidf_train_features = tfidf_vectorizer.fit_transform(bow_train_features)
tfidf_test_features = tfidf_vectorizer.transform(bow_test_features)    

# características averaged word vector
avg_wv_train_features = averaged_word_vectorizer(norm_train_corpus)                
avg_wv_test_features = averaged_word_vectorizer(norm_test_corpus)      

# características tfidf weighted averaged word vector
word_tfidf_map = {key:value for (key, value) in zip(bow_vectorizer.get_feature_names_out(), tfidf_vectorizer.idf_)}

tfidf_wv_train_features = tfidf_weighted_averaged_word_vectorizer(norm_train_corpus, word_tfidf_map)

tfidf_wv_test_features = tfidf_weighted_averaged_word_vectorizer(norm_test_corpus, word_tfidf_map)

In [ ]:
bow_train_features.shape

In [ ]:
bow_vectorizer.get_feature_names_out()

In [ ]:
tfidf_train_features.shape

In [ ]:
tfidf_vectorizer.get_feature_names_out()

In [ ]:
avg_wv_train_features.shape

In [ ]:
tfidf_wv_train_features.shape

## 5. CLASIFICADOR
Aplicamos distintos clasificadores a cada modelo para ver cuál funciona mejor con nuestros datos. Primero definimos unas funciones para entrenar y medir el rendimiento de los clasificadores:

In [ ]:
def get_metrics(true_labels, predicted_labels):
    """Calculamos distintas métricas sobre el
    rendimiento del modelo. Devuelve un diccionario
    con los parámetros medidos"""
    
    return {
        'Accuracy': np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        3),
        'Precision': np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted',
                                               zero_division=0),
                        3),
    'Recall': np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted',
                                               zero_division=0),
                        3),
    'F1 Score': np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted',
                                               zero_division=0),
                        3)}
                        

def train_predict_evaluate_model(classifier, 
                                 train_features, train_labels, 
                                 test_features, test_labels):
    """Función que entrena un modelo de clasificación sobre
    un conjunto de entrenamiento, lo aplica sobre un conjunto
    de test y devuelve la predicción sobre el conjunto de test
    y las métricas de rendimiento"""
    # genera modelo    
    classifier.fit(train_features, train_labels)
    # predice usando el modelo sobre test
    predictions = classifier.predict(test_features) 
    # evalúa rendimiento de la predicción   
    metricas = get_metrics(true_labels=test_labels, 
                predicted_labels=predictions)
    return predictions, metricas     

Vamos a entrenar sobre el conjunto de train y evaluamos en el conjunto de test. Guardamos métricas en una lista y resultados en otra para mostrar resumen.

In [ ]:
modelLR = LogisticRegression(solver='liblinear')
modelNB = GaussianNB() #var_smoothing=1e-9
modelSVM = SGDClassifier(loss='hinge', max_iter=1000)
modelRBFSVM = SVC(gamma='scale', C=2)

modelos = [('Logistic Regression', modelLR),
           ('Naive Bayes', modelNB),
           ('Linear SVM', modelSVM),
           ('Gauss kernel SVM', modelRBFSVM)]

metricas = []
resultados = []

# Modelos con características BoW
bow_train_features_ = bow_train_features.toarray()
bow_test_features_ = bow_test_features.toarray()
for m, clf in modelos:
    prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                           train_features=bow_train_features_,
                                           train_labels=train_labels,
                                           test_features=bow_test_features_,
                                           test_labels=test_labels)
    metrica['modelo']=f'{m} BoW'
    resultados.append(prediccion)
    metricas.append(metrica)
    
# Modelos con características TF-IDF
tfidf_train_features_ = tfidf_train_features.toarray()
tfidf_test_features_ = tfidf_test_features.toarray()
for m, clf in modelos:
    prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                           train_features=tfidf_train_features_,
                                           train_labels=train_labels,
                                           test_features=tfidf_test_features_,
                                           test_labels=test_labels)
    metrica['modelo']=f'{m} tfidf'
    resultados.append(prediccion)
    metricas.append(metrica)

# Modelos con características averaged word vectors
avg_wv_train_features_ = avg_wv_train_features
avg_wv_test_features_ = avg_wv_test_features
for m, clf in modelos:
    prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                           train_features=avg_wv_train_features_,
                                           train_labels=train_labels,
                                           test_features=avg_wv_test_features_,
                                           test_labels=test_labels)
    metrica['modelo']=f'{m} averaged'
    resultados.append(prediccion)
    metricas.append(metrica)

# Modelos con características tfidf weighted averaged word vectors
tfidf_wv_train_features_ = tfidf_wv_train_features
tfidf_wv_test_features_ = tfidf_wv_test_features
for m, clf in modelos:
    prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                           train_features=tfidf_wv_train_features_,
                                           train_labels=train_labels,
                                           test_features=tfidf_wv_test_features_,
                                           test_labels=test_labels)
    metrica['modelo']=f'{m} tfidf wv'
    resultados.append(prediccion)
    metricas.append(metrica)

Conviertimos la lista de métricas en un DataFrame para observar sus valores:

In [ ]:
metricas = pd.DataFrame(metricas)
metricas

In [ ]:
metricas.to_csv(prediccion_r+'/metricas.csv', header=True, index=False)

Ordenamos las métricas por `accuracy` y muestra el mejor resultado:

In [ ]:
metricas = metricas.sort_values('Accuracy',ascending=False)
metricas.head(1)

Mejoramos el `accuracy` a partir del juego de parametros

In [ ]:
modelos = metricas['modelo'].tolist()

mejor = modelos[0]
sep = mejor.split(' ')



if sep[len(sep)-1]== "wv":
    mo = ' '.join(sep[:len(sep)-2])
    
else:
    mo = ' '.join(sep[:len(sep)-1])

print(mo)
    

datos = sep[len(sep)-1]
print(datos)

metricas2 = []
resultados = []



# DATOS
if datos == 'Bow':
    train_features_ = bow_train_features.toarray()
    test_features_ = bow_test_features.toarray()
    
if datos == 'tfidf':
    train_features_ = tfidf_train_features.toarray()
    test_features_ = tfidf_test_features.toarray()
    
if datos == 'averaged':
    train_features_ = avg_wv_train_features
    test_features_ = avg_wv_test_features
    
else: # tfidf wv
    train_features_ = tfidf_wv_train_features
    test_features_ = tfidf_wv_test_features




# MODELOS

if mo == 'Logistic Regression':
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

    sol = ["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"]
    
    for i in sol:
        modelLR = LogisticRegression(solver=i)
        prediccion, metrica = train_predict_evaluate_model(classifier=modelLR,
                                            train_features=train_features_,
                                            train_labels=train_labels,
                                            test_features=test_features_,
                                            test_labels=test_labels)
        metrica['modelo']=f'{i} - Logistic Regression ' + datos
        resultados.append(prediccion)
        metricas2.append(metrica)
    
if mo == 'Naive Bayes':
    # https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html
    var_smo = [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9,1e-10,1e-11,1e-12,1e-13,1e-14,1e-15]
    
    for i in var_smo:
        modelNB = GaussianNB(var_smoothing=i)
        prediccion, metrica = train_predict_evaluate_model(classifier=modelLR,
                                            train_features=train_features_,
                                            train_labels=train_labels,
                                            test_features=test_features_,
                                            test_labels=test_labels)
        metrica['modelo']=f'{i} - Naive Bayes ' + datos
        resultados.append(prediccion)
        metricas2.append(metrica)
    
if mo == 'Linear SVM':
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html
    loss = ["hinge", "log_loss", "log", "modified_huber", "squared_hinge", "perceptron", "squared_error", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"]

    for i in loss:
        modelSVM = SGDClassifier(loss=i, max_iter=1000)
        prediccion, metrica = train_predict_evaluate_model(classifier=modelLR,
                                            train_features=train_features_,
                                            train_labels=train_labels,
                                            test_features=test_features_,
                                            test_labels=test_labels)
        metrica['modelo']=f'{i} - Linear SVM ' + datos
        resultados.append(prediccion)
        metricas2.append(metrica)

    
else: # Gauss kernel SVM 
    # https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
    gamma = ["scale", "auto"]

    for i in gamma:
        modelRBFSVM = SVC(gamma=i, C=2)
        prediccion, metrica = train_predict_evaluate_model(classifier=modelLR,
                                            train_features=train_features_,
                                            train_labels=train_labels,
                                            test_features=test_features_,
                                            test_labels=test_labels)
        metrica['modelo']=f'{i} - Gauss kernel SVM ' + datos
        resultados.append(prediccion)
        metricas2.append(metrica)


Conviertimos la lista de métricas en un DataFrame para observar sus valores:

In [ ]:
metricas3 = pd.DataFrame(metricas2)
metricas3

Ordenamos las métricas por `accuracy` y muestra el mejor resultado:

In [ ]:
metricas3 = metricas3.sort_values('Accuracy',ascending=False)
metricas3.head(1)

## 6. PRODUCCIÓN [PENDIENTE DE CAMBIAR] 
Cogemos el modelo que mejor funciona y lo aplicamos de forma manual

Dividimos el conjunto total de muestras (`corpus` y `labels`) en entrenamiento y test (30%) y entrenamos el mejor modelo obtenido, para ver si se mejoran los resultados:

In [ ]:
## MEJOR MODELO
train_corpus, test_corpus, train_labels, test_labels = train_test_split(corpus, labels, test_size = 0.30)


In [ ]:
#Normalizamos
norm_train_corpus = list(map(normaliza, train_corpus))
norm_test_corpus = list(map(normaliza, test_corpus))



In [ ]:
# Características averaged word vector
avg_wv_train_features = averaged_word_vectorizer(norm_train_corpus)                
avg_wv_test_features = averaged_word_vectorizer(norm_test_corpus)      
avg_wv_train_features_ = avg_wv_train_features
avg_wv_test_features_ = avg_wv_test_features

In [ ]:
# Modelos con características
modelRBFSVM = SVC(gamma='scale', C=2)
prediccion, metrica = train_predict_evaluate_model(classifier=modelRBFSVM,
                                           train_features=avg_wv_train_features_,
                                           train_labels=train_labels,
                                           test_features=avg_wv_test_features_,
                                           test_labels=test_labels)



In [ ]:
print('Modelo Gauss kernel SVM averaged con gamma scale: ')
for i in metrica:
    print('- ',i, ': ', metrica[i])

print('\nPredicciones:')
print(prediccion)

In [ ]:
pd.unique(prediccion)